In [25]:
import torch
import torch.nn.functional as F


B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
# x is a 3D tensor with shape (B, T, C)

x.shape

torch.Size([4, 8, 2])

infromation should only from from previous time steps to future ones. in theory a simple way to do this is to add the average of all preceeding tokens. 

In [5]:
# we want x[b, t] = mean_{i <=t } x [b, i]
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # everything up to and including t'th token... and C (all 2d info from those tokens)
        xbow[b, t] = torch.mean(xprev, dim=0) # mean across the time dimension

In [7]:
torch.tril(torch.ones(3, 3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [ ]:
# use matmul to be more efficient

torch.manual_seed(42)

a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


c[0, 0] is a[1, :] dot b[:, 1]... so on and so forth. 

with tril, the zeroes are ignored

In [18]:
wei = torch.tril(torch.ones(T, T))  # lower triangular matrix
wei = wei / wei.sum(1, keepdim=True)

xbow2 = wei @ x  # matrix multiplication
# its like this (T, T) x (B, T, C)... pytorch will come put in a batch dimension making it
# (B, T, T) x (B, T, C) -> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [28]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))  # fill upper triangular part with -inf
wei = F.softmax(wei, dim=-1)  # apply softmax to the lower triangular matrix

print(wei)

xbow3 = wei @ x  # matrix multiplication

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


In [31]:
# using softmax
# ensures that output sums to 1
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))  # fill upper triangular part with -inf
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [35]:
import torch.nn as nn


torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)  # (B, T, head_size)
q = query(x)  # (B, T, head_size)

wei = q @ k.transpose(-2, -1)  # (B, T, head_size) @ (B, head_size, T) -> (B, T, T)

# wei = torch.zeros((T, T))
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v  # (B, T, T) @ (B, T, head_size) -> (B, T, head_size)

out.shape

torch.Size([4, 8, 16])